In [18]:
import os
from dotenv import load_dotenv

import bs4
import tiktoken
import numpy as np
from operator import itemgetter
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate,FewShotChatMessagePromptTemplate
from langchain.load import dumps, loads
from langchain_community.utilities import BingSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_groq import ChatGroq




In [5]:
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
GROQ_API_KEY= os.getenv('GROQ_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ["BING_SUBSCRIPTION_KEY"] =os.getenv('BING_SUBSCRIPTION_KEY')
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")



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


In [23]:
search = BingSearchAPIWrapper(k=1)
output = search.run("what os open banking?")
output

'<b>Open</b> <b>Banking</b> has paved the way for a number of useful &#39;money management&#39; websites and apps which use your financial data to provide you with a personalised service or recommendations on ways to save based on your habits. These include apps which automatically save for you, provide you with tailor-made budgeting tips, or display all your ...'

In [24]:
search1 = DuckDuckGoSearchRun()
search2 = DuckDuckGoSearchResults()

output1=search1.run("what os open banking?")
output2=search2.run("what os open banking?")

print(output1)
print(output2)

Open banking can give customers more control over their financial information and provide new services and applications. For nonfinancial companies, this shift means they are able to offer customized financial services to their customers, make more data-driven decisions, and innovate in payments and account management. Open Banking: A system that provides a user with a network of financial institutions' data through the use of application programming interfaces, better known as APIs. The Open Banking Standard ... Benefits of open banking. Open banking is poised to create a wave of innovation in the financial sector. One of the most significant benefits is the ability to gain a more comprehensive view of a consumer's financial situation. With a deeper view of consumer cashflow data and access to actionable insights, you can improve your underwriting ... Open banking transforms the financial landscape by promoting competition, innovation, and customer-centric services. It gives individua

In [25]:
llm=ChatGroq(model_name="llama3-8b-8192")


que = "what os open banking?"

output3= llm.invoke(que)
output3

AIMessage(content='Open Banking is a set of regulations and technologies that enables secure and standardized communication between financial institutions, fintech companies, and consumers. It aims to promote competition, innovation, and consumer choice in the financial services industry.\n\nThe key aspects of Open Banking are:\n\n1. **Data sharing**: Financial institutions are required to share customer data with authorized third-party providers, such as fintech companies, upon request.\n2. **API-based access**: Financial institutions must provide Application Programming Interfaces (APIs) for secure and standardized access to customer data and payment systems.\n3. **Security and authentication**: Strong authentication and encryption measures are implemented to ensure the confidentiality, integrity, and availability of customer data.\n4. **Consumer consent**: Customers must explicitly consent to share their data with third-party providers.\n5. **Compliance**: Financial institutions mus

In [6]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, 
    chunk_overlap=50)


splits = text_splitter.split_documents(blog_docs)

vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [27]:
# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""

In [28]:
prompt_perspectives = ChatPromptTemplate.from_template(template)

In [29]:
generate_queries = (
    prompt_perspectives 
    | ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [30]:
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

In [31]:
question = "What is Self-Reflection for LLM agents?"
retriever_chain = generate_queries | retriever.map() | get_unique_union
docs = retriever_chain.invoke({"question":question})

In [32]:
docs

[Document(page_content='Fig. 2.  Examples of reasoning trajectories for knowledge-intensive tasks (e.g. HotpotQA, FEVER) and decision-making tasks (e.g. AlfWorld Env, WebShop). (Image source: Yao et al. 2023).\nIn both experiments on knowledge-intensive tasks and decision-making tasks, ReAct works better than the Act-only baseline where Thought: … step is removed.\nReflexion (Shinn & Labash 2023) is a framework to equips agents with dynamic memory and self-reflection capabilities to improve reasoning skills. Reflexion has a standard RL setup, in which the reward model provides a simple binary reward and the action space follows the setup in ReAct where the task-specific action space is augmented with language to enable complex reasoning steps. After each action $a_t$, the agent computes a heuristic $h_t$ and optionally may decide to reset the environment to start a new trial depending on the self-reflection results.\n\nFig. 3. Illustration of the Reflexion framework. (Image source: Shi

In [33]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

In [34]:
final_rag_chain=(
    {"context":retriever_chain,
     "question":itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Self-reflection for LLM agents is a vital aspect that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.'

In [35]:
output = final_rag_chain.invoke({"question":question}).strip()
print(output)

Self-reflection for LLM agents allows them to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.


## Part 6: RAG-Fusion




Docs:

* https://github.com/langchain-ai/langchain/blob/master/cookbook/rag_fusion.ipynb?ref=blog.langchain.dev

Blog / repo: 

* https://towardsdatascience.com/forget-rag-the-future-is-rag-fusion-1147298d8ad1

### Prompt

In [36]:
# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [37]:
generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [38]:
def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

In [39]:
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})


print(len(docs))
print(docs)

4
[(Document(page_content='Self-Reflection#\nSelf-reflection is a vital aspect that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.\nReAct (Yao et al. 2023) integrates reasoning and acting within LLM by extending the action space to be a combination of task-specific discrete actions and the language space. The former enables LLM to interact with the environment (e.g. use Wikipedia search API), while the latter prompting LLM to generate reasoning traces in natural language.\nThe ReAct prompt template incorporates explicit steps for LLM to think, roughly formatted as:\nThought: ...\nAction: ...\nObservation: ...\n... (Repeated many times)', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}), 0.13224043715846995), (Document(page_content='Fig. 2.  Examples of reasoning trajectories for knowledge-intensive tasks (e.g.

In [40]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Self-reflection for LLM agents is a vital aspect that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.'

## Part 7: Decomposition

In [9]:
# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [10]:
llm = ChatOpenAI(temperature=0)

#chian
generate_queries_decomposition = prompt_decomposition | llm | StrOutputParser() | (lambda x:x.split("\n"))

In [11]:
# Run
question = "What are the main components of an LLM-powered autonomous agent system?"

questions = generate_queries_decomposition.invoke({"question":question})

In [45]:
questions

['1. What is LLM technology and how does it work in autonomous agent systems?',
 '2. What are the specific components that make up an LLM-powered autonomous agent system?',
 '3. How do the main components of an LLM-powered autonomous agent system interact with each other to enable autonomous behavior?']

In [5]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [12]:
def format_qa_pair(question, answer):
    """Format Q and A pair"""
    formatted_string=""
    formatted_string+=f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()


llm = ChatOpenAI(temperature=0)



In [19]:
q_a_pairs = ""
for q in questions:
    
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [20]:


answer

"The main components of an LLM-powered autonomous agent system, namely planning, memory, and tool use, interact with each other in a cohesive manner to enable autonomous behavior. \n\n1. Planning: The agent utilizes planning to break down complex tasks into smaller subgoals for efficient handling. This involves task decomposition, where the agent can utilize techniques like Chain of Thought (CoT) or Tree of Thoughts to transform big tasks into manageable steps. Additionally, the agent can engage in self-reflection to refine strategies based on past actions, thereby improving the quality of its decision-making and task execution.\n\n2. Memory: The agent's memory plays a crucial role in enabling autonomous behavior. Short-term memory is utilized for in-context learning, allowing the agent to adapt and learn from immediate experiences. Long-term memory, on the other hand, enables the retention and recall of vast amounts of information over extended periods. This is often achieved through 

In [9]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

In [10]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: 2+2
AI: 4
Human: 2+3
AI: 5


In [13]:

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

chain = final_prompt| ChatOpenAI(temperature=0)

chain.invoke({"input":"what's square of a triangle?"})

In [15]:

examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]


example_promt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_promt,
    examples=examples
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [16]:
generate_queries_step_back = prompt | ChatOpenAI(temperature=0) | StrOutputParser()
question = "what is task decomposition for llm agents?"
generate_queries_step_back.invoke({"question":question})

'what is the process of breaking down tasks for llm agents?'

In [17]:
# Response prompt 
response_prompt_template = """You are an expert of world knowledge. 
I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context 
if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""

In [19]:
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    |response_prompt
    |ChatOpenAI(temperature=0)
    |StrOutputParser()
)

chain.invoke({"question":question})

'Task decomposition for LLM agents involves breaking down large tasks into smaller, more manageable subgoals. This process enables LLM-powered autonomous agents to efficiently handle complex tasks by dividing them into smaller steps. Task decomposition can be achieved through various methods, such as prompting the LLM with specific instructions like "Steps for XYZ" or "What are the subgoals for achieving XYZ," using task-specific instructions tailored to the nature of the task, or incorporating human inputs to guide the decomposition process.\n\nOne common technique for task decomposition is the Chain of Thought (CoT), which prompts the model to "think step by step" and decompose hard tasks into simpler steps. Another approach, the Tree of Thoughts, extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of thought steps and generating multiple thoughts per step. This allows for a more comprehensive breakdown of the problem into manageable comp